# Testing out the spotify and spotipy api's!


In [10]:
# Import required libraries
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import os
import pandas as pd

print("Libraries imported successfully!")

Libraries imported successfully!


In [11]:
# Load environment variables from .env file
load_dotenv()

# Get Spotify API credentials from environment
SPOTIFY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIFY_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')
SPOTIFY_REDIRECT_URI = os.getenv('SPOTIFY_REDIRECT_URI', 'http://localhost:8888/callback')

# Verify credentials are loaded
if not SPOTIFY_CLIENT_ID or not SPOTIFY_CLIENT_SECRET:
    print("ERROR: Missing Spotify credentials in .env file!")
    print("Please ensure SPOTIFY_CLIENT_ID and SPOTIFY_CLIENT_SECRET are set.")
else:
    print("Credentials loaded successfully!")
    print(f"Client ID: {SPOTIFY_CLIENT_ID[:10]}...")
    print(f"Redirect URI: {SPOTIFY_REDIRECT_URI}")

Credentials loaded successfully!
Client ID: 4f226b6e1e...
Redirect URI: http://127.0.0.1:8501/


In [12]:
# Define OAuth scopes for the permissions we need
# See: https://developer.spotify.com/documentation/web-api/concepts/scopes
SCOPES = [
    'user-read-recently-played',   # Access recently played tracks
    'user-top-read',                # Access top artists and tracks
    'user-library-read',            # Access saved tracks and albums
    'playlist-read-private',        # Access private playlists
    'playlist-read-collaborative',  # Access collaborative playlists
    'user-read-playback-state',     # Read current playback state
    'user-read-currently-playing',  # Read currently playing track
]

# Join scopes into a single space-separated string
scope_string = ' '.join(SCOPES)

print(f"Requesting {len(SCOPES)} permissions:")
for scope in SCOPES:
    print(f"  - {scope}")

Requesting 7 permissions:
  - user-read-recently-played
  - user-top-read
  - user-library-read
  - playlist-read-private
  - playlist-read-collaborative
  - user-read-playback-state
  - user-read-currently-playing


In [13]:
# Create SpotifyOAuth object for authentication
# This will handle the OAuth flow and token management
auth_manager = SpotifyOAuth(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET,
    redirect_uri=SPOTIFY_REDIRECT_URI,
    scope=scope_string,
    cache_path='.cache_notebook',  # Separate cache file for notebook
    open_browser=True              # Automatically open browser for auth
)

print("SpotifyOAuth manager created!")
print(f"Cache file: .cache_notebook")
print(f"\nReady to authenticate...")

SpotifyOAuth manager created!
Cache file: .cache_notebook

Ready to authenticate...


In [14]:
# Create authenticated Spotify client
# This will trigger the OAuth flow:
# 1. Opens browser to Spotify login
# 2. User authorizes the app
# 3. Redirects to localhost with auth code
# 4. Copy the full redirect URL and paste it when prompted
sp = spotipy.Spotify(auth_manager=auth_manager)

print("Authentication successful!")
print("Spotify client created and ready to use.")

Authentication successful!
Spotify client created and ready to use.


In [15]:
# Test authentication by fetching user profile
user_profile = sp.current_user()

print("Current User Profile:")
print(f"  Display Name: {user_profile.get('display_name', 'N/A')}")
print(f"  User ID: {user_profile.get('id', 'N/A')}")
print(f"  Email: {user_profile.get('email', 'N/A')}")
print(f"  Country: {user_profile.get('country', 'N/A')}")
print(f"  Product: {user_profile.get('product', 'N/A')}")
print(f"  Followers: {user_profile.get('followers', {}).get('total', 0)}")
print(f"  Profile URL: {user_profile.get('external_urls', {}).get('spotify', 'N/A')}")

print("\nAuthentication verified! You can now start making API calls.")

Current User Profile:
  Display Name: nico_diferd
  User ID: nico_diferd
  Email: N/A
  Country: N/A
  Product: N/A
  Followers: 13
  Profile URL: https://open.spotify.com/user/nico_diferd

Authentication verified! You can now start making API calls.


## Example API Calls

Now that authentication is set up, here are some common API calls you can make:

In [16]:
# Example 1: Get recently played tracks
recent_tracks = sp.current_user_recently_played(limit=5)

print("Recently Played Tracks:")
print("-" * 80)
for idx, item in enumerate(recent_tracks['items'], 1):
    track = item['track']
    played_at = item['played_at']
    print(f"{idx}. {track['name']} - {track['artists'][0]['name']}")
    print(f"   Played at: {played_at}")
    print()

Recently Played Tracks:
--------------------------------------------------------------------------------
1. CAN'T SAY - Travis Scott
   Played at: 2025-11-21T00:47:36.254Z

2. Folded - Kehlani
   Played at: 2025-11-21T00:44:32.925Z

3. Buck 50 - DJ Scheme
   Played at: 2025-11-21T00:17:22.908Z

4. Yessirskiii - Lil Uzi Vert
   Played at: 2025-11-21T00:14:42.314Z

5. I'm Still - Juice WRLD
   Played at: 2025-11-20T23:55:48.798Z



In [17]:
# Example 2: Get top tracks (short term = last 4 weeks)
top_tracks = sp.current_user_top_tracks(limit=5, time_range='short_term')

print("Top Tracks (Last 4 Weeks):")
print("-" * 80)
for idx, track in enumerate(top_tracks['items'], 1):
    artists = ', '.join([artist['name'] for artist in track['artists']])
    print(f"{idx}. {track['name']} - {artists}")
    print(f"   Popularity: {track['popularity']}/100")
    print()

Top Tracks (Last 4 Weeks):
--------------------------------------------------------------------------------
1. Flicker of Light - Lola Young
   Popularity: 60/100

2. Good Game - Dominic Fike
   Popularity: 50/100

3. Your Teeth In My Neck - Kali Uchis
   Popularity: 68/100

4. Folded - Kehlani
   Popularity: 89/100

5. Just A Stranger (feat. Steve Lacy) - Kali Uchis, Steve Lacy
   Popularity: 65/100



In [18]:
# Example 3: Get audio features for a track
# NOTE: sp.audio_features() may not work reliably - use alternate approach below
# Using the first track from top tracks
if top_tracks['items']:
    track_id = top_tracks['items'][0]['id']
    track_name = top_tracks['items'][0]['name']
    
    try:
        audio_features = sp.audio_features([track_id])[0]
        
        print(f"Audio Features for: {track_name}")
        print("-" * 80)
        print(f"  Danceability: {audio_features['danceability']:.3f}")
        print(f"  Energy: {audio_features['energy']:.3f}")
        print(f"  Valence (happiness): {audio_features['valence']:.3f}")
        print(f"  Acousticness: {audio_features['acousticness']:.3f}")
        print(f"  Instrumentalness: {audio_features['instrumentalness']:.3f}")
        print(f"  Speechiness: {audio_features['speechiness']:.3f}")
        print(f"  Tempo: {audio_features['tempo']:.1f} BPM")
        print(f"  Loudness: {audio_features['loudness']:.1f} dB")
        print(f"  Key: {audio_features['key']}")
        print(f"  Mode: {'Major' if audio_features['mode'] == 1 else 'Minor'}")
        print(f"  Time Signature: {audio_features['time_signature']}/4")
    except Exception as e:
        print(f"ERROR: sp.audio_features() failed - {e}")
        print("See alternate approach below using sp.track()")

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3etbPFMXnAuShtcImz4UXW with Params: {} returned 403 due to None


ERROR: sp.audio_features() failed - http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=3etbPFMXnAuShtcImz4UXW:
 None, reason: None
See alternate approach below using sp.track()


## Alternate Approach: Using sp.track() for Metadata

Since `sp.audio_features()` may not work, let's explore what metadata we can get from `sp.track()` instead.

In [19]:
# Step 1: Get recent 10 tracks listened to
recent_limit = 10
recent_tracks = sp.current_user_recently_played(limit=recent_limit)

print(f"Retrieved {len(recent_tracks['items'])} recently played tracks")
print("-" * 80)

# Extract track IDs for iteration
track_ids = []
for idx, item in enumerate(recent_tracks['items'], 1):
    track = item['track']
    track_id = track['id']
    track_ids.append(track_id)
    print(f"{idx}. {track['name']} by {track['artists'][0]['name']}")
    print(f"   Track ID: {track_id}")
    print(f"   Played at: {item['played_at']}")
    print()

print(f"\nCollected {len(track_ids)} track IDs for detailed analysis")

Retrieved 10 recently played tracks
--------------------------------------------------------------------------------
1. CAN'T SAY by Travis Scott
   Track ID: 27a1mYSG5tYg7dmEjWBcmL
   Played at: 2025-11-21T00:47:36.254Z

2. Folded by Kehlani
   Track ID: 0bxPRWprUVpQK0UFcddkrA
   Played at: 2025-11-21T00:44:32.925Z

3. Buck 50 by DJ Scheme
   Track ID: 7nD5MiXspW533xaREDma2N
   Played at: 2025-11-21T00:17:22.908Z

4. Yessirskiii by Lil Uzi Vert
   Track ID: 59JWp4PjZ9TRM8cmtaDYB1
   Played at: 2025-11-21T00:14:42.314Z

5. I'm Still by Juice WRLD
   Track ID: 19m3E0nMZobyA93AxMXC32
   Played at: 2025-11-20T23:55:48.798Z

6. Now Or Never - Bonus Track by Kendrick Lamar
   Track ID: 4KjnaUNYPwGnJjoeTFlt91
   Played at: 2025-11-20T23:53:08.318Z

7. Myself by NAV
   Track ID: 05nbZ1xxVNwUTcGwLbp7CN
   Played at: 2025-11-20T23:49:04.029Z

8. Down Low - YDG Remix by it's murph
   Track ID: 20iODoi8eZUC1sBS5C01GF
   Played at: 2025-11-20T23:46:42.256Z

9. Myself by NAV
   Track ID: 05nbZ1xxVN

In [20]:
# Step 2: Iterate through track IDs and get detailed metadata using sp.track()
print("Detailed Metadata for Each Track:")
print("=" * 80)

track_metadata_list = []

for idx, track_id in enumerate(track_ids, 1):
    try:
        # Get full track details
        track_details = sp.track(track_id)
        
        print(f"\n{idx}. {track_details['name']}")
        print("-" * 80)
        
        # Basic Info
        print("BASIC INFO:")
        print(f"  Track Name: {track_details['name']}")
        print(f"  Track ID: {track_details['id']}")
        print(f"  URI: {track_details['uri']}")
        print(f"  Spotify URL: {track_details['external_urls']['spotify']}")
        
        # Artists
        print("\nARTISTS:")
        for artist in track_details['artists']:
            print(f"  - {artist['name']} (ID: {artist['id']})")
        
        # Album
        print("\nALBUM:")
        print(f"  Album Name: {track_details['album']['name']}")
        print(f"  Album Type: {track_details['album']['album_type']}")
        print(f"  Release Date: {track_details['album']['release_date']}")
        print(f"  Total Tracks: {track_details['album']['total_tracks']}")
        
        # Metrics
        print("\nMETRICS:")
        print(f"  Popularity: {track_details['popularity']}/100")
        print(f"  Duration: {track_details['duration_ms']/1000:.2f} seconds")
        print(f"  Explicit: {track_details['explicit']}")
        print(f"  Track Number: {track_details['track_number']}")
        print(f"  Disc Number: {track_details['disc_number']}")
        
        # Additional
        print("\nADDITIONAL:")
        print(f"  Is Playable: {track_details.get('is_playable', 'N/A')}")
        print(f"  Is Local: {track_details.get('is_local', False)}")
        if 'preview_url' in track_details and track_details['preview_url']:
            print(f"  Preview URL: Available")
        else:
            print(f"  Preview URL: Not available")
        
        # Store for later analysis
        track_metadata_list.append({
            'track_id': track_details['id'],
            'track_name': track_details['name'],
            'artist_name': track_details['artists'][0]['name'],
            'artist_id': track_details['artists'][0]['id'],
            'album_name': track_details['album']['name'],
            'album_type': track_details['album']['album_type'],
            'release_date': track_details['album']['release_date'],
            'popularity': track_details['popularity'],
            'duration_ms': track_details['duration_ms'],
            'explicit': track_details['explicit'],
            'track_number': track_details['track_number']
        })
        
    except Exception as e:
        print(f"\nERROR for track {track_id}: {e}")

print(f"\n\nCollected metadata for {len(track_metadata_list)} tracks")

Detailed Metadata for Each Track:

1. CAN'T SAY
--------------------------------------------------------------------------------
BASIC INFO:
  Track Name: CAN'T SAY
  Track ID: 27a1mYSG5tYg7dmEjWBcmL
  URI: spotify:track:27a1mYSG5tYg7dmEjWBcmL
  Spotify URL: https://open.spotify.com/track/27a1mYSG5tYg7dmEjWBcmL

ARTISTS:
  - Travis Scott (ID: 0Y5tJX1MQlPlqiwlOH1tJY)

ALBUM:
  Album Name: ASTROWORLD
  Album Type: album
  Release Date: 2018-08-03
  Total Tracks: 17

METRICS:
  Popularity: 76/100
  Duration: 198.41 seconds
  Explicit: True
  Track Number: 13
  Disc Number: 1

ADDITIONAL:
  Is Playable: N/A
  Is Local: False
  Preview URL: Not available

2. Folded
--------------------------------------------------------------------------------
BASIC INFO:
  Track Name: Folded
  Track ID: 0bxPRWprUVpQK0UFcddkrA
  URI: spotify:track:0bxPRWprUVpQK0UFcddkrA
  Spotify URL: https://open.spotify.com/track/0bxPRWprUVpQK0UFcddkrA

ARTISTS:
  - Kehlani (ID: 0cGUm45nv7Z6M6qdXYQGTX)

ALBUM:
  Album Na

In [21]:
# Step 3: Convert to DataFrame for analysis
df_tracks = pd.DataFrame(track_metadata_list)

print("Track Metadata DataFrame:")
print("=" * 80)
print(f"Shape: {df_tracks.shape}")
print(f"\nColumns: {list(df_tracks.columns)}")
print(f"\nFirst few rows:")
print(df_tracks.head())

print(f"\n\nData Types:")
print(df_tracks.dtypes)

print(f"\n\nBasic Statistics:")
print(df_tracks[['popularity', 'duration_ms']].describe())

Track Metadata DataFrame:
Shape: (10, 11)

Columns: ['track_id', 'track_name', 'artist_name', 'artist_id', 'album_name', 'album_type', 'release_date', 'popularity', 'duration_ms', 'explicit', 'track_number']

First few rows:
                 track_id   track_name   artist_name               artist_id  \
0  27a1mYSG5tYg7dmEjWBcmL    CAN'T SAY  Travis Scott  0Y5tJX1MQlPlqiwlOH1tJY   
1  0bxPRWprUVpQK0UFcddkrA       Folded       Kehlani  0cGUm45nv7Z6M6qdXYQGTX   
2  7nD5MiXspW533xaREDma2N      Buck 50     DJ Scheme  1m7LSAMIB1BErIHYSOn32W   
3  59JWp4PjZ9TRM8cmtaDYB1  Yessirskiii  Lil Uzi Vert  4O15NlyKLIASxsJ0PrXPfz   
4  19m3E0nMZobyA93AxMXC32    I'm Still    Juice WRLD  4MCBfE4596Uoi2O4DtmEMz   

                                     album_name album_type release_date  \
0                                    ASTROWORLD      album   2018-08-03   
1                                        Folded     single   2025-06-11   
2                               FAMILY (Deluxe)      album   2021-01-

In [22]:
# Step 4: Potential Metrics from Track Metadata
print("POTENTIAL METRICS FOR TOP SONGS ANALYSIS:")
print("=" * 80)

# 1. Popularity Distribution
print("\n1. POPULARITY ANALYSIS:")
print(f"   Average Popularity: {df_tracks['popularity'].mean():.2f}/100")
print(f"   Max Popularity: {df_tracks['popularity'].max()}")
print(f"   Min Popularity: {df_tracks['popularity'].min()}")
print(f"   Median Popularity: {df_tracks['popularity'].median()}")

# 2. Duration Analysis
print("\n2. DURATION ANALYSIS:")
avg_duration_seconds = df_tracks['duration_ms'].mean() / 1000
print(f"   Average Duration: {avg_duration_seconds:.2f} seconds ({avg_duration_seconds/60:.2f} minutes)")
print(f"   Shortest Track: {df_tracks['duration_ms'].min()/1000:.2f} seconds")
print(f"   Longest Track: {df_tracks['duration_ms'].max()/1000:.2f} seconds")

# 3. Artist Diversity
print("\n3. ARTIST DIVERSITY:")
unique_artists = df_tracks['artist_name'].nunique()
print(f"   Unique Artists: {unique_artists} out of {len(df_tracks)} tracks")
print(f"   Artist Diversity: {(unique_artists/len(df_tracks)*100):.1f}%")
print(f"   Top Artists:")
for artist, count in df_tracks['artist_name'].value_counts().head(3).items():
    print(f"      - {artist}: {count} tracks")

# 4. Album Type Distribution
print("\n4. ALBUM TYPE DISTRIBUTION:")
for album_type, count in df_tracks['album_type'].value_counts().items():
    print(f"   {album_type}: {count} ({count/len(df_tracks)*100:.1f}%)")

# 5. Explicit Content
print("\n5. EXPLICIT CONTENT:")
explicit_count = df_tracks['explicit'].sum()
print(f"   Explicit Tracks: {explicit_count} ({explicit_count/len(df_tracks)*100:.1f}%)")
print(f"   Clean Tracks: {len(df_tracks)-explicit_count} ({(len(df_tracks)-explicit_count)/len(df_tracks)*100:.1f}%)")

# 6. Release Year Analysis (extract from release_date)
print("\n6. RELEASE YEAR ANALYSIS:")
df_tracks['release_year'] = df_tracks['release_date'].str[:4]
print(f"   Years Represented: {df_tracks['release_year'].nunique()}")
print(f"   Year Range: {df_tracks['release_year'].min()} - {df_tracks['release_year'].max()}")
print(f"   Most Common Years:")
for year, count in df_tracks['release_year'].value_counts().head(3).items():
    print(f"      - {year}: {count} tracks")

print("\n" + "=" * 80)
print("These metrics can help identify:")
print("  - Listening preferences (mainstream vs niche)")
print("  - Genre diversity (via artist variety)")
print("  - Temporal preferences (new releases vs classics)")
print("  - Content preferences (explicit vs clean)")
print("  - Track length preferences (short vs long songs)")

POTENTIAL METRICS FOR TOP SONGS ANALYSIS:

1. POPULARITY ANALYSIS:
   Average Popularity: 67.50/100
   Max Popularity: 89
   Min Popularity: 57
   Median Popularity: 66.5

2. DURATION ANALYSIS:
   Average Duration: 218.03 seconds (3.63 minutes)
   Shortest Track: 189.52 seconds
   Longest Track: 256.01 seconds

3. ARTIST DIVERSITY:
   Unique Artists: 9 out of 10 tracks
   Artist Diversity: 90.0%
   Top Artists:
      - NAV: 2 tracks
      - Travis Scott: 1 tracks
      - Kehlani: 1 tracks

4. ALBUM TYPE DISTRIBUTION:
   album: 8 (80.0%)
   single: 2 (20.0%)

5. EXPLICIT CONTENT:
   Explicit Tracks: 6 (60.0%)
   Clean Tracks: 4 (40.0%)

6. RELEASE YEAR ANALYSIS:
   Years Represented: 7
   Year Range: 2012 - 2025
   Most Common Years:
      - 2018: 2 tracks
      - 2021: 2 tracks
      - 2017: 2 tracks

These metrics can help identify:
  - Listening preferences (mainstream vs niche)
  - Genre diversity (via artist variety)
  - Temporal preferences (new releases vs classics)
  - Content p

## Bonus: Get Artist Details for Genre Information

Track metadata doesn't include genres, but we can get genres from artist details using `sp.artist()`

In [23]:
# Get artist details for genre information
unique_artist_ids = df_tracks['artist_id'].unique()

print(f"Fetching details for {len(unique_artist_ids)} unique artists...")
print("=" * 80)

artist_details_list = []

for artist_id in unique_artist_ids[:5]:  # Limit to first 5 to avoid too many API calls
    try:
        artist = sp.artist(artist_id)
        
        print(f"\nArtist: {artist['name']}")
        print("-" * 40)
        print(f"  Genres: {', '.join(artist['genres']) if artist['genres'] else 'No genres listed'}")
        print(f"  Popularity: {artist['popularity']}/100")
        print(f"  Followers: {artist['followers']['total']:,}")
        print(f"  Spotify URL: {artist['external_urls']['spotify']}")
        
        artist_details_list.append({
            'artist_id': artist['id'],
            'artist_name': artist['name'],
            'genres': artist['genres'],
            'popularity': artist['popularity'],
            'followers': artist['followers']['total']
        })
        
    except Exception as e:
        print(f"ERROR for artist {artist_id}: {e}")

print(f"\n\nCollected details for {len(artist_details_list)} artists")

Fetching details for 9 unique artists...

Artist: Travis Scott
----------------------------------------
  Genres: rap
  Popularity: 90/100
  Followers: 41,378,957
  Spotify URL: https://open.spotify.com/artist/0Y5tJX1MQlPlqiwlOH1tJY

Artist: Kehlani
----------------------------------------
  Genres: No genres listed
  Popularity: 80/100
  Followers: 8,732,690
  Spotify URL: https://open.spotify.com/artist/0cGUm45nv7Z6M6qdXYQGTX

Artist: DJ Scheme
----------------------------------------
  Genres: No genres listed
  Popularity: 54/100
  Followers: 300,503
  Spotify URL: https://open.spotify.com/artist/1m7LSAMIB1BErIHYSOn32W

Artist: Lil Uzi Vert
----------------------------------------
  Genres: melodic rap
  Popularity: 83/100
  Followers: 19,324,755
  Spotify URL: https://open.spotify.com/artist/4O15NlyKLIASxsJ0PrXPfz

Artist: Juice WRLD
----------------------------------------
  Genres: melodic rap, emo rap
  Popularity: 86/100
  Followers: 42,807,618
  Spotify URL: https://open.spot

In [24]:
# Summary: Additional Metrics from Artist Data
print("ADDITIONAL METRICS FROM ARTIST DATA:")
print("=" * 80)

if artist_details_list:
    df_artists = pd.DataFrame(artist_details_list)
    
    print("\n1. GENRE DISTRIBUTION:")
    # Flatten genre lists
    all_genres = []
    for genres in df_artists['genres']:
        all_genres.extend(genres)
    
    if all_genres:
        from collections import Counter
        genre_counts = Counter(all_genres)
        print(f"   Total unique genres: {len(genre_counts)}")
        print(f"   Top genres:")
        for genre, count in genre_counts.most_common(5):
            print(f"      - {genre}: {count}")
    else:
        print("   No genre data available for these artists")
    
    print("\n2. ARTIST POPULARITY:")
    print(f"   Average Artist Popularity: {df_artists['popularity'].mean():.2f}/100")
    print(f"   Most Popular Artist: {df_artists.loc[df_artists['popularity'].idxmax(), 'artist_name']} ({df_artists['popularity'].max()}/100)")
    
    print("\n3. FOLLOWER ANALYSIS:")
    print(f"   Average Followers: {df_artists['followers'].mean():,.0f}")
    print(f"   Most Followed Artist: {df_artists.loc[df_artists['followers'].idxmax(), 'artist_name']} ({df_artists['followers'].max():,} followers)")
    
    print("\n" + "=" * 80)
    print("COMBINED METRICS POSSIBLE:")
    print("  - Genre diversity score (# unique genres / # artists)")
    print("  - Mainstream score (based on artist popularity & followers)")
    print("  - Niche explorer score (low popularity but high diversity)")
    print("  - Genre affinity (top 3 genres by frequency)")
    print("  - Artist loyalty (repeated artists in listening history)")
else:
    print("No artist data collected")

ADDITIONAL METRICS FROM ARTIST DATA:

1. GENRE DISTRIBUTION:
   Total unique genres: 3
   Top genres:
      - melodic rap: 2
      - rap: 1
      - emo rap: 1

2. ARTIST POPULARITY:
   Average Artist Popularity: 78.60/100
   Most Popular Artist: Travis Scott (90/100)

3. FOLLOWER ANALYSIS:
   Average Followers: 22,508,905
   Most Followed Artist: Juice WRLD (42,807,618 followers)

COMBINED METRICS POSSIBLE:
  - Genre diversity score (# unique genres / # artists)
  - Mainstream score (based on artist popularity & followers)
  - Niche explorer score (low popularity but high diversity)
  - Genre affinity (top 3 genres by frequency)
  - Artist loyalty (repeated artists in listening history)


## Helpful Resources

**Spotipy Documentation:**
- [Spotipy Docs](https://spotipy.readthedocs.io/en/latest/)
- [API Reference](https://spotipy.readthedocs.io/en/latest/#api-reference)

**Common Time Ranges:**
- `short_term`: Last 4 weeks
- `medium_term`: Last 6 months
- `long_term`: Several years

**Common Limits:**
- Most endpoints support `limit` parameter (max 50)
- Use `offset` for pagination

**Tip:** You can batch audio features requests for up to 100 tracks at once:
```python
track_ids = [track['id'] for track in tracks]
audio_features = sp.audio_features(track_ids)
```

---

## Your Workspace

Start building below! The `sp` client is authenticated and ready to use.

In [30]:
results = sp.search(q='artist:Drake', type='track', limit=2)
display(results)

{'tracks': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=2&query=artist%3ADrake&type=track',
  'limit': 2,
  'next': 'https://api.spotify.com/v1/search?offset=2&limit=2&query=artist%3ADrake&type=track',
  'offset': 0,
  'previous': None,
  'total': 100,
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'},
       'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4',
       'id': '3TVXtAsR1Inumwj472S9r4',
       'name': 'Drake',
       'type': 'artist',
       'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'DO',
      'DE',
      'EC',
      'EE',
      'SV',
      'FI',
      'FR',
      'GR',
      'GT',
      'HN',
      'HK',
      'HU',
      'IS',
      'IE',
   